In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from PIL import Image, ImageDraw, ImageFont
import json
import random
import gradio as gr
import queue

import clip
import os
from tqdm import tqdm
from utils.init_model import model, preprocess, load_model, device
from utils.transform_image import draw_text_with_new_lines
from dataset.dataset import all_attributes
from utils.initialize_font_data import retrieve_font_path, gray_scale_image_file_dir
from evals.evaluate_tools import predict_cos_sim

from cj_fonts import inclusive_fonts, fifty_fonts
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager

from torchvision.transforms.functional import pil_to_tensor, to_pil_image

exclusive_attributes:  ['capitals', 'cursive', 'display', 'italic', 'monospace', 'serif']


In [2]:
char_size = 150
font_dir = '../gwfonts'
cj_font_dir = '../all-fonts'
annotation_dir = '../attributeData/outputs'
fox_text = 'The quick\nbrown fox\njumps over\nthe lazy dog'
font_paths = [os.path.join(font_dir, f) for f in os.listdir(font_dir)]
cj_font_paths = [os.path.join(cj_font_dir, f) for f in os.listdir(cj_font_dir)]

# add font
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

for font in font_manager.findSystemFonts(cj_font_dir):
    font_manager.fontManager.addfont(font)

ttf_list = font_manager.fontManager.ttflist

predicted_attributes = json.load(open('../attributeData/predicted_cj_font_attribute.json', 'r'))
attribute_to_indexes = {attribute: all_attributes.index(attribute) for attribute in all_attributes}

def compare_two_annotations(name1, name2, target_attribute, font_num=50):
    target_attribute = target_attribute.replace(' ', '_')
    annotation1 = json.load(open(os.path.join(annotation_dir, f'{name1}-{target_attribute}-{font_num}.json'), 'r'))
    annotation2 = json.load(open(os.path.join(annotation_dir, f'{name2}-{target_attribute}-{font_num}.json'), 'r'))
    correct_count = 0
    for (font1, font2, attribute, choice1), (font3, font4, attribute2, choice2) in zip(annotation1, annotation2):
        if not ((font1 == font3) and (font2 == font4) and (attribute == attribute2)):
            raise ValueError('annotation not match')
        if choice1 == choice2:
            correct_count += 1
    print(correct_count / len(annotation1))

def compare_with_clip(name, model, text, target_attribute, font_num=50):
    target_attribute_file_name = target_attribute.replace(' ', '_')
    annotation = json.load(open(os.path.join(annotation_dir, f'{name}-{target_attribute_file_name}-{font_num}.json'), 'r'))
    correct_count = 0
    for i, (font1, font2, attribute, choice) in enumerate(annotation):
        font_path1 = retrieve_font_path(font1)
        font_path2 = retrieve_font_path(font2)
        font1 = ImageFont.truetype(font_path1, char_size)
        font2 = ImageFont.truetype(font_path2, char_size)
        cos1, image1 = predict_cos_sim(font1, attribute, model=model, text=text)
        cos2, image2 = predict_cos_sim(font2, attribute, model=model, text=text)
        if choice == 'Font A':
            if cos1 >= cos2:
                correct_count += 1
        elif choice == 'Font B':
            if cos1 <= cos2:
                correct_count += 1
        else:
            raise ValueError('invalid choice')
    return correct_count / len(annotation)

In [25]:
checkpoint_path = 'model_checkpoints/new_best_fox_negative_91011_1011_use_weight_image_file_dir_ex.pt'
checkpoint_path = 'model_checkpoints/best_ViT-B_32_cnn_based_vae_loss_weight_3.0_vae_loss_kl_weight0.001_res_64_9101191011_batch64_aug200_lower_bound_of_scale0.35_use_negative_lr1e-05-0.1.pt'
#checkpoint_path = 'model_checkpoints/67891011_891011_multiple_3_1000_image_file_dir_aug_ex.pt'
#checkpoint_path = 'model_checkpoints/new_best_fox_negative_91011_91011_multiple_3_1000_aug_ex.pt'
# checkpoint_path = None

model = load_model(model, checkpoint_path)

In [26]:
text = '春夏\n秋冬'
name = 'Akihiro_Kiuchi'
target_attribute = 'robust'
compare_with_clip(name, model, text, target_attribute)

0.5490196078431373

In [23]:
names = ['Akihiro_Kiuchi', 'TakahiroHosokawa', 'SodaiFuruoka']
attributes = ['thin', 'calm', 'sloppy', 'sharp', 'technical', 'traditional', 'Japanese style', 'robust']

for attribute in attributes:
    s = 0
    for name in names:
        tmp_result = compare_with_clip(name, model, text, attribute)
        #print(name, attribute, tmp_result)
        s += tmp_result
    print(attribute, s / len(names))

thin 0.7866666666666667
calm 0.5098039215686274
sloppy 0.522875816993464
sharp 0.6993464052287582
technical 0.6862745098039215
traditional 0.6274509803921569
Japanese style 0.6339869281045751
robust 0.6732026143790849


In [28]:
names = ['Akihiro_Kiuchi', 'TakahiroHosokawa', 'SodaiFuruoka']
attributes = ['thin', 'calm', 'sloppy', 'sharp', 'technical', 'traditional', 'Japanese style', 'robust']
attributes = ['thin', 'calm', 'sloppy', 'sharp', 'technical']
attributes = ['traditional', 'Japanese style', 'robust']

s = 0
for attribute in attributes:
    tmp_s = 0
    for name in names:
        tmp_result = compare_with_clip(name, model, text, attribute)
        #print(name, attribute, tmp_result)
        tmp_s += tmp_result
    print(attribute, tmp_s / len(names))
    s += tmp_s / len(names)
print(s / len(attributes))

traditional 0.6339869281045752
Japanese style 0.6143790849673203
robust 0.5555555555555556
0.6013071895424836


In [28]:
def choose_font_pairs_same_apper_times_with_queue(font_paths, appear_times=8, target_font_num=50):
    sorted_font_paths = sorted(font_paths)

    random.seed(123)
    random.shuffle(sorted_font_paths)
    font_paths = sorted_font_paths[:target_font_num] * appear_times

    font_paths_queue = queue.Queue()
    for font_path in font_paths:
        font_paths_queue.put(font_path)
    flag = False
    count = 0
    # extract two font paths from font_paths_queue
    while True:
        if font_paths_queue.qsize() < 2:
            break
        if count > 1000:
            break

        font_path2 = font_paths_queue.get()
        if not flag:
            font_path1 = font_paths_queue.get()
        flag = False
        if font_path1 != font_path2:
            print(font_path1, font_path2)
            yield font_path1, font_path2
        else:
            count += 1
            font_paths_queue.put(font_path2)
            flag = True

In [34]:
count = 0
target_attribute = 'thin'
text = '春夏秋冬\n朝昼晩'
text = '春夏\n秋冬'
user_name = 'Yuki_Tatsukawa'
font_num = 50
annotation = json.load(open(os.path.join(annotation_dir, f'{user_name}-{target_attribute}-{font_num}.json'), 'r'))

default_font1 = ImageFont.truetype(retrieve_font_path(annotation[0][0]), char_size)
default_font2 = ImageFont.truetype(retrieve_font_path(annotation[0][1]), char_size)
default_human_choice = annotation[0][3]
default_width = int(char_size * len(text) / (text.count('\n') + 1))
default_height = (char_size + 30) * (text.count('\n') + 1)
cos1, image1 = predict_cos_sim(default_font1, target_attribute, text=fox_text)
cos2, image2 = predict_cos_sim(default_font2, target_attribute, text=fox_text)
clip_prediction = 'Font A' if cos1 >= cos2 else 'Font B'

def builder():
    global count
    assert count < len(annotation)
    font1 = ImageFont.truetype(retrieve_font_path(annotation[count][0]), char_size)
    font2 = ImageFont.truetype(retrieve_font_path(annotation[count][1]), char_size)
    cos1, image1 = predict_cos_sim(font1, target_attribute, text=fox_text)
    cos2, image2 = predict_cos_sim(font2, target_attribute, text=fox_text)
    clip_prediction = 'Font A' if cos1 >= cos2 else 'Font B'
    cj_image1 = draw_text_with_new_lines(text, font1, default_width, default_height)
    cj_image2 = draw_text_with_new_lines(text, font2, default_width, default_height)
    human_choice = annotation[count][3]
    count += 1
    return cj_image1, cj_image2, human_choice, clip_prediction

with gr.Blocks() as demo:

    with gr.Row():
        name = gr.Text(value=user_name, label='Name', interactive=True)
    with gr.Row():
        image1 = gr.Image(value=draw_text_with_new_lines(text, default_font1, default_width, default_height), label='Font A', interactive=False)
        image2 = gr.Image(value=draw_text_with_new_lines(text, default_font2, default_width, default_height), label='Font B', interactive=False)

    with gr.Row():
        #target_attribute_text = gr.Text(value=target_attribute, label='Target Attribute', interactive=False, visible=False)
        #attribute_markdown = gr.Markdown(value=f'<div style="font-size: 50px; text-align: center">Which font is more {target_attribute}?</div>', label='Target Attribute', interactive=False)
        attribute_markdown = gr.Text(value=f'{target_attribute}', label='Target Attribute', interactive=False)
        #target_attribute = gr.Text(value=choose_random_target_attribute(target_attributes), label='Target Attribute', interactive=False)

    with gr.Row():
        check1 = gr.Radio(choices=['Font A', 'Font B'], value = default_human_choice, label=f'Chosen by user', interactive=False)
        check2 = gr.Radio(choices=['Font A', 'Font B'], value = clip_prediction, label=f'Chosen by CLIP', interactive=False)

    with gr.Row():
        next_button  = gr.Button(value='Next', interactive=True)
        #next_button.click(builder, inputs=[check, target_attribute], outputs=[target_attribute, image1, image2])
    next_button.click(builder, inputs=[], outputs=[image1, image2, check1, check2], show_progress=False)

demo.launch(debug=True, share=False)



Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [31]:
annotation

[['JP_Ronde-B_square', 'SanariFontB001', 'thin', 'Font B'],
 ['ZenOldMincho-Black', 'HanyiSentyPastel', 'thin', 'Font A'],
 ['crayon_1-1', 'SentyCaramel', 'thin', 'Font B'],
 ['kiloji_d', 'irohamaru-Medium', 'thin', 'Font A'],
 ['Tanugo-TTF-Regular', 'YuseiMagic-Regular', 'thin', 'Font A'],
 ['chogokubosogothic_5', 'g_pencilkaisho_free', 'thin', 'Font A'],
 ['HanyiSentyPastel', 'ゆず ポップ A [M] Light', 'thin', 'Font B'],
 ['HachiMaruPop-Regular', 'crayon_1-1', 'thin', 'Font A'],
 ['KFhimaji', 'Corporate-Logo-Medium-ver3', 'thin', 'Font B'],
 ['kiloji_d', 'Kazesawa-ExtraLight', 'thin', 'Font B'],
 ['Corporate-Logo-Medium-ver3', 'chinese-handwriting-style', 'thin', 'Font B'],
 ['SentyGoldSand', 'Chalk-S-JP', 'thin', 'Font B'],
 ['g_comickoin_freeL', '35', 'thin', 'Font A'],
 ['g_comickoin_freeB', 'ZCOOLXiaoWei-Regular', 'thin', 'Font B'],
 ['KodomoRounded', 'M+A1_heavy-10-1.2', 'thin', 'Font A'],
 ['NasuM-Bold-20200227', 'SentyGoldSand', 'thin', 'Font A'],
 ['JP_Ronde-B_square', 'ロゴたいぷ